# _IMDB Explore Data_

> In this project I will present a basic guide for exploratory data analysis using Python Language, SQL Language and SQLite Database. I will use real publicly available data about movies on <a href="https://www.imdb.com/interfaces/">IMDB</a>. 

> When applying Exploratory Data Analysis I will answer these 10 questions: 

> - 1- What Are the Most Common Movie Categories on IMDB?
> - 2- What is the number of titles by genre?
> - 3- What is the Median Rating of Films by Gender?
> - 4- What is the Median Rating of Films in relation to the year of release?
> - 5- What is the number of films rated by genre in relation to the year of release?
> - 6- What is the Longest Time Movie? Calculate Percentiles.
> - 7- What is the Relationship Between Duration and Gender?
> - 8- What is the number of films produced by country?
> - 9- What are the Top 10 Best Movies?
> - 10- What Are the Top 10 Worst Movies? 

### _Installing and Loading Packages_

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Python version used in this notebook:', python_version())

Python version used in this notebook: 3.8.5


In [2]:
# Installing imdb package
!pip install -q imdb-sqlite

In [3]:
# Installing pycountry package
# https://pypi.org/project/pycountry/
!pip install -q pycountry

In [4]:
# Imports
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

### _Loading Data_

>First we have to download the ratings data from imdb 

In [5]:
%%time
!imdb-sqlite

Wall time: 313 ms


2021-12-23 22:36:58,045 DB already exists: (imdb.db). Refusing to modify. Exiting


In [6]:
# Connecting in the database
conn = sqlite3.connect("imdb.db")

In [7]:
# Extracting table list
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)
type(tables)

pandas.core.frame.DataFrame

In [8]:
# Show results
#We have 6 tables in this database:
tables

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes
5,ratings


In [9]:
# Convert dataframe to list
tables = tables["Table_Name"].values.tolist()

In [10]:
# Let's extract the schema of each table 
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    result = pd.read_sql_query(query, conn)
    print("Schema of: {}".format(table))
    display(result)
    print("-"*100)
    print("\n")

Schema of: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Schema of: akas


...

----------------------------------------------------------------------------------------------------


Schema of: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




> Description of each table: 
> * ratings: User ratings.
> * episodes: The film can be: A television series, a documentary, etc.. organized into seasons and episodes
> * crew: Team that participated in the production of the film.
> * akas: Filme region and language. 
> * titles: Movie names.
> * people: Actors and actresses who participated in the film.


## Now let's start exploring the data

#### _1- What Are the Most Common Movie Categories on IMDB?_

In [13]:
#Creating SQL query
query1 = '''SELECT type, COUNT(*) AS COUNT FROM titles GROUP BY type'''

In [29]:
#Executing SQL query
res1 = pd.read_sql_query(query1, conn)

In [30]:
#Showing result
display(res1)

,type,COUNT
0,movie,597005
1,short,847120
2,tvEpisode,6364611
3,tvMiniSeries,41156
4,tvMovie,134077
5,tvPilot,2
6,tvSeries,218094
7,tvShort,10421
8,tvSpecial,35303
9,video,255797


In [31]:
#Calculating the percentage for each type
res1['percent'] = (res1['COUNT'] / res1['COUNT'].sum()) * 100

In [32]:
display(res1)

,type,COUNT,percent
0,movie,597005,6.995980
1,short,847120,9.926942
2,tvEpisode,6364611,74.583444
3,tvMiniSeries,41156,0.482285
4,tvMovie,134077,1.571176
5,tvPilot,2,0.000023
6,tvSeries,218094,2.555726
7,tvShort,10421,0.122118
8,tvSpecial,35303,0.413697
9,video,255797,2.997547


In [33]:
#Let's create a graph with only 4 categories, because the human brain doesn't process much more than that simultaneously.
#The three categories with the most titles. 

#Create an empty dictionary
others = {}

#Filter the percentage in 5% and add the total
others['COUNT'] = res1[res1['percent'] < 5]['COUNT'].sum()

#Record the percentage
others['percent'] = res1[res1['percent'] < 5]['percent'].sum()

#adjust the name
others['type'] = 'others'

In [34]:
#Show others
others

{'COUNT': 724808, 'percent': 8.49363406340906, 'type': 'others'}

In [35]:
#Filter the dataframe where percent > 5 
res1 = res1[res1['percent'] >= 5]

#Appendo with dataframe from other categories
res1 = res1.append(others, ignore_index = True)

#Order the result
res1 = res1.sort_values(by = 'COUNT', ascending = False)

#Show
res1.head()

,type,COUNT,percent
2,tvEpisode,6364611,74.583444
1,short,847120,9.926942
3,others,724808,8.493634
0,movie,597005,6.995980


#### _2- What is the number of titles by genre?_

#### _3- What is the Median Rating of Films by Gender?_

#### _4- What is the Median Rating of Films in relation to the year of release?_

#### _5- What is the number of films rated by genre in relation to the year of release?_

#### _6- What is the Longest Time Movie? Calculate Percentiles._

#### _7- What is the Relationship Between Duration and Gender?_

#### _8- What is the number of films produced by country?_

#### _9- What are the Top 10 Best Movies?_

#### _10- What Are the Top 10 Worst Movies?_